# Initialize environment
> NOTE: Make sure you have a .env file loaded

In [2]:
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv

# Get Configuration Settings
load_dotenv()

client = AzureOpenAI(api_key=os.getenv('AOAI_KEY'), azure_endpoint=os.getenv('AOAI_ENDPOINT'), api_version=os.environ.get("AOAI_VERSION","2023-03-15-preview"))

In [3]:
#https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
response = client.chat.completions.create(
    model="gpt-35-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "What's the difference between garbanzo beans and chickpeas?"}
    ]
)

print(response)

print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-8irqquY2tNjGPhCNZQ8QIRfKN3VgB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Garbanzo beans and chickpeas are actually the same thing - they are just known by different names in different parts of the world. In the United States, they are typically referred to as chickpeas, while in many other countries such as Mexico and Spain, they are called garbanzo beans. So whether you call them chickpeas or garbanzo beans, you're talking about the same type of legume!", role='assistant', function_call=None, tool_calls=None))], created=1705704128, model='gpt-35-turbo', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=87, prompt_tokens=38, total_tokens=125))
Garbanzo beans and chickpeas are actually the same thing - they are just known by different names in different parts of the world. In the United States, they are typically referred to as chickpeas, while in many other c

Example from the documentation around Chat GPT and using tokens from [here](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions)

In [4]:
conversation=[{"role": "system", "content": "You are a helpful assistant."}]

while(True):
    user_input = input()      
    if user_input == "exit" or user_input == "quit" or user_input == "":
        break
    conversation.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="gpt-35-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
        messages = conversation
    )

    conversation.append({"role": "assistant", "content": response.choices[0].message.content})
    print("\n" + response.choices[0].message.content + "\n")


Why did the tomato turn red? Because it saw the salad dressing!


Why did the cookie go to the doctor? Because it was feeling crummy.



In [ ]:
system_message = {"role": "system", "content": "You are a helpful assistant."}
max_response_tokens = 250
token_limit= 4096
conversation=[]
conversation.append(system_message)

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

while(True):
    user_input = input("")
    if user_input == "exit" or user_input == "quit" or user_input == "":
        break
             
    conversation.append({"role": "user", "content": user_input})
    conv_history_tokens = num_tokens_from_messages(conversation)

    while (conv_history_tokens+max_response_tokens >= token_limit):
        del conversation[1] 
        conv_history_tokens = num_tokens_from_messages(conversation)
        
    response = client.chat.completions.create(
        model="gpt-35-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
        messages = conversation,
        temperature=.7,
        max_tokens=max_response_tokens,
    )

    conversation.append({"role": "assistant", "content": response.choices[0].message.content})
    print("\n" + response.choices[0].message.content + "\n")

# Bot for reading content from a web site

In [ ]:
url=input("Enter a url for a web site")
conversation=[{"role": "system", "content": f"You are a helpful assistant that knows alot about {url}. Read the webside and answer questions."}]

while(True):
    user_input = input()      
    if user_input == "exit" or user_input == "quit" or user_input == "":
        break
    conversation.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="gpt-35-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
        messages = conversation
    )

    conversation.append({"role": "assistant", "content": response.choices[0].message.content})
    print("\n" + response.choices[0].message.content + "\n")

# Standard prompts

In [ ]:
prompt="""The CSS code for a color like a blue sky at dusk:

background-color: #"""
response=client.completions.create(prompt=prompt,
    model='text-davinci-003',
    max_tokens=64, 
    temperature=0, 
    top_p=1, 
    frequency_penalty=0.0, 
    presence_penalty=0.0, 
    stop=[";"])

print(f"{response.choices[0].text}")

In [ ]:
prompt="Write a tagline for an ice cream shop."
response=client.completions.create(
    prompt=prompt,
    mocel='text-davinci-003'
)

print(f"{response.choices[0].text}")


In [ ]:
prompt="""
The following is a list of companies and the categories they fall into:

Apple, Facebook, Fedex

Apple
Category:
"""

response = client.completions.create(
  model='text-davinci-003',
  prompt=prompt,
  temperature=0,
  max_tokens=64,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

print(f"{response.choices[0].text}")

# All about SQL Helper

In [5]:
prompt="""
### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
#Create a SQL query for A query to list the names of the departments which employed more than 10 employees in the last 3 months
SELECT
"""

response = client.completions.create(
  model='code-davinci-002',
  prompt=prompt,
  temperature=0,
  max_tokens=150,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
    stop=["#",";"]
)

print(f"{response.choices[0].text}")

NameError: name 'openai' is not defined

## Bot to write SQL for multiple questions

In [ ]:
# Same thing but will ONLY generate the SQL statement
prompt="""
Postgres SQL tables, with their properties:
Employee(id, name, department_id)
Department(id, name, address)
Salary_Payments(id, employee_id, amount, date)

#Create a SQL query for "{0}"
"""

while(True):
    user_input = input()      
    if user_input == "exit" or user_input == "quit" or user_input == "":
        break

    sql = prompt.format(user_input)

    response = client.completions.create(
    model='code-davinci-002',
    prompt=sql,
    temperature=0,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
      stop=["#",";"]
    )

    print(f"{response.choices[0].text}")

## This using GPT model and gets more conversational
Do you like the conversation? You can easily parse the SQL returned with LangChain or other coding to then turnaround and execute the SQL for the end users

In [ ]:
# Chat SQL with ChatGPT
prompt="""
Tables and columns:
Employee(id, name, department_id)
Department(id, name, address)
Salary_Payments(id, employee_id, amount, date)

You are a bot that generates SQL code based on the "Tables and columns". Return standard SQL queries that a user could then run in a tool like SQL Server Management Studio
"""

conversation=[{"role": "system", "content": prompt}]

while(True):
    user_input = input()      
    if user_input == "exit" or user_input == "quit" or user_input == "":
        break
    conversation.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="gpt-35-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
        messages = conversation
    )

    conversation.append({"role": "assistant", "content": response.choices[0].message.content})
    print("\n" + response.choices[0].message.content + "\n")

In [ ]:
conversation=[{"role": "system", "content": "You are a helpful assistant."}]

while(True):
    user_input = input()      
    if user_input == "exit" or user_input == "quit" or user_input == "":
        break
    conversation.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="gpt-35-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
        messages = conversation
    )

    conversation.append({"role": "assistant", "content": response.choices[0].message.content})
    print("\n" + response.choices[0].message.content + "\n")

In [6]:
prompt="""
Convert movie titles into emoji.

Back to the Future: 👨👴🚗🕒 
Batman: 🤵🦇 
Transformers: 🚗🤖 
Star Wars:
"""

response = client.completions.create(
  model='text-davinci-003',
  prompt=prompt,
  temperature=0.8,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["\n"]
)

print(f"{response.choices[0].text}")


✨🌌⭐️💫


In [ ]:
prompt="""
class Log:
    def __init__(self, path):
        dirname = os.path.dirname(path)
        os.makedirs(dirname, exist_ok=True)
        f = open(path, "a+")

        # Check that the file is newline-terminated
        size = os.path.getsize(path)
        if size > 0:
            f.seek(size - 1)
            end = f.read(1)
            if end != "\n":
                f.write("\n")
        self.f = f
        self.path = path

    def log(self, event):
        event["_event_id"] = str(uuid.uuid4())
        json.dump(event, self.f)
        self.f.write("\n")

    def state(self):
        state = {"complete": set(), "last": None}
        for line in open(self.path):
            event = json.loads(line)
            if event["type"] == "submit" and event["success"]:
                state["complete"].add(event["id"])
                state["last"] = event
        return state

#
Here's what the above class is doing:
1.
"""

response = client.completions.create(
  model='code-davinci-002',
  prompt=prompt,
  temperature=0,
  max_tokens=64,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["#"]
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
Q: Who is Batman?
A: Batman is a fictional comic book character.

Q: What is torsalplexity?
A: ?

Q: What is Devz9?
A: ?

Q: Who is George Lucas?
A: George Lucas is American film director and producer famous for creating Star Wars.

Q: What is the capital of California?
A: Sacramento.

Q: What orbits the Earth?
A: The Moon.

Q: Who is Fred Rickerson?
A: ?

Q: What is an atom?
A: An atom is a tiny particle that makes up everything.

Q: Who is Alvan Muntz?
A: ?

Q: What is Kozar-09?
A: ?

Q: How many moons does Mars have?
A: Two, Phobos and Deimos.

Q: What's a language model?
A:
"""

response = client.completions.create(
  model='text-davinci-003',
  prompt=prompt,
  temperature=0,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
##### Fix bugs in the below function
 
### Buggy Python
import Random
a = random.randint(1,12)
b = random.randint(1,12)
for i in range(10):
    question = "What is "+a+" x "+b+"? "
    answer = input(question)
    if answer = a*b
        print (Well done!)
    else:
        print("No.")
    
### Fixed Python
"""

response = client.completions.create(
  model='code-davinci-002',
  prompt=prompt,
  temperature=0,
  max_tokens=182,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["###"]
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
You: How do I combine arrays?
JavaScript chatbot: You can use the concat() method.
You: How do you make an alert appear after 10 seconds?
JavaScript chatbot
"""

response = client.completions.create(
  model='code-davinci-002',
  prompt=prompt,
  temperature=0,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.5,
  presence_penalty=0.0,
  stop=["You:"]
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
Extract the airport codes from this text:

Text: "I want to fly from Los Angeles to Miami."
Airport codes: LAX, MIA

Text: "I want to fly from Orlando to Boston"
Airport codes:
"""

response = client.completions.create(
  model='text-davinci-003',
  prompt=prompt,
  max_tokens=60,
  temperature=0,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["\n"]
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
You: What have you been up to?
Friend: Watching old movies.
You: Did you watch anything interesting?
Friend:
"""

response = client.completions.create(
    model='text-davinci-003',
    prompt=prompt,
    temperature=0.5,
    max_tokens=60,
    top_p=1.0,
    frequency_penalty=0.5,
    presence_penalty=0.0,
    stop=["You:"]
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
Brainstorm some ideas combining VR and fitness:
"""

response = client.completions.create(
    model='text-davinci-003',
    prompt=prompt,
    temperature=0.6,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=1,
    presence_penalty=1
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
Create a list of 8 questions for my interview with a science fiction author:
"""

response = client.completions.create(
    model='text-davinci-003',
    prompt=prompt,
    temperature=0.5,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0,
    presence_penalty=0
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
Create an outline for an essay about Nikola Tesla and his contributions to technology:
"""

response = client.completions.create(
    model='text-davinci-003',
    prompt=prompt,
    temperature=0.3,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0,
    presence_penalty=0
)

print(f"{response.choices[0].text}")

In [ ]:
prompt="""
What are 5 key points I should know when studying Ancient Rome?
"""

response = client.completions.create(
    model='text-davinci-003',
    prompt=prompt,
    temperature=0.3,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0,
    presence_penalty=0
)

print(f"{response.choices[0].text}")

Example from the documentation around Chat GPT and using tokens from [here](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions)

In [ ]:
import tiktoken

system_message = {"role": "system", "content": "You are a helpful assistant."}
max_response_tokens = 250
token_limit= 4096
conversation=[]
conversation.append(system_message)

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

while(True):
    user_input = input("")
    if user_input == "exit" or user_input == "quit" or user_input == "":
        break
             
    conversation.append({"role": "user", "content": user_input})
    conv_history_tokens = num_tokens_from_messages(conversation)

    while (conv_history_tokens+max_response_tokens >= token_limit):
        del conversation[1] 
        conv_history_tokens = num_tokens_from_messages(conversation)
        
    response = client.chat.completions.create(
        model="gpt-35-turbo", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
        messages = conversation,
        temperature=.7,
        max_tokens=max_response_tokens,
    )

    conversation.append({"role": "assistant", "content": response.choices[0].message.content})
    print("\n" + response.choices[0].message.content + "\n")